<a href="https://colab.research.google.com/github/Rishabh5752/textattack/blob/main/Textattack_Tensorflow_Attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install textattack[tensorflow]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached textattack-0.3.8-py3-none-any.whl (418 kB)
  Using cached bert_score-0.3.13-py3-none-any.whl (61 kB)
  Using cached flair-0.12.2-py3-none-any.whl (373 kB)
  Using cached language_tool_python-2.7.1-py3-none-any.whl (34 kB)
  Using cached lemminflect-0.2.3-py3-none-any.whl (769 kB)
  Using cached lru_dict-1.2.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (28 kB)
  Using cached datasets-2.4.0-py3-none-any.whl (365 kB)
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached terminaltables-3.1.10-py2.py3-none-any.whl (15 kB)
  Using cached word2number-1.1.zip (9.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached num2words-0.5.12-py3-none-any.whl (125 kB)
  Using cached pinyin-0.4.0.tar.gz (3.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached OpenHowNet-2.0-py3-none-any.whl (18 kB)
   

In [3]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"],
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Version:  2.9.1
Eager mode:  True
Hub version:  0.13.0
GPU is NOT AVAILABLE


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete7VWDHZ/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete7VWDHZ/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
30/30 [==============================] - 2s 54ms/step - loss: 0.7834 - accuracy: 0.5619 - val_loss: 0.6961 - val_accuracy: 0.5968
Epoch 2/40
30/30 [===========

In [4]:
import numpy as np
import torch

from textattack.models.wrappers import ModelWrapper

class CustomTensorFlowModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        text_array = np.array(text_input_list)
        preds = self.model(text_array).numpy()
        logits = torch.exp(-torch.tensor(preds))
        logits = 1 / (1 + logits)
        logits = logits.squeeze(dim=-1)
        # Since this model only has a single output (between 0 or 1),
        # we have to add the second dimension.
        final_preds = torch.stack((1-logits, logits), dim=1)
        return final_preds


textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
model_wrapper = CustomTensorFlowModelWrapper(model)

from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019
from textattack import Attacker

dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)
attack = PWWSRen2019.build(model_wrapper)

attacker = Attacker(attack, dataset)
attacker.attack_dataset()

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

--------------------------------------------- Result 3 ---------------------------------------------
[[Positive (97%)]] --> [[Negative (59%)]]

it's like a " big chill " reunion of the baader-meinhof gang , only these [[guys]] are more harmless pranksters than political activists .

it's like a " big chill " reunion of the baader-meinhof gang , only these [[roast]] are more harmless pranksters than political activists .


--------------------------------------------- Result 4 ---------------------------------------------
[[Negative (70%)]] --> [[[SKIPPED]]]

the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .


--------------------------------------------- Result 5 ---------------------------------------------
[[Negative (98%)]] --> [[[SKIPPED]]]

red dragon " never cuts corners .


--------------------------------------------- Result 6 ---------------------------------------------
[[Positive (99%)]] --> 

[Succeeded / Failed / Skipped / Total] 5 / 0 / 3 / 8:  80%|████████  | 8/10 [00:01<00:00,  5.38it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (91%)]]

[[throws]] in enough clever and unexpected [[twists]] to make the formula feel fresh .

[[drop]] in enough clever and unexpected [[construction]] to make the formula feel fresh .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (97%)]] --> [[Negative (69%)]]

weighty and ponderous but every [[bit]] as filling as the treat of the title .

weighty and ponderous but every [[bite]] as filling as the treat of the title .




[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:01<00:00,  5.81it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (70%)]]

a [[real]] audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital [[bed]] or insurance company office .

a [[material]] audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital [[screw]] or insurance company office .


--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (92%)]] --> [[[SKIPPED]]]

generates an enormous feeling of empathy for its characters .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 60.0%  |
|